# **Modelo listo para usarse**

**Verificar si hay GPU disponible**

In [1]:
!nvidia-smi

Tue Mar  5 23:34:21 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.61                 Driver Version: 551.61         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:0A:00.0  On |                  N/A |
|  0%   45C    P0             46W /  285W |    2429MiB /  16376MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Instalar librería con el modelo YOLO

In [2]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable


In [1]:

import numpy as np
import cv2
from ultralytics import YOLO
import torch

**Realizar inferencia**

In [4]:


# Verifica si hay una GPU disponible y, si es así, configura PyTorch para usarla
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
# Carga la imagen
img = cv2.imread("D:/edurb/Escritorio/Escuela/MODULAR/Program/STA/MODEL/frame_255.jpg")
img_szd = cv2.resize(img, (640, 480))

# Convierte el array de NumPy a un tensor de PyTorch
img_szd = torch.from_numpy(img_szd)

# Reorganiza las dimensiones para que sean BCHW en lugar de HWC
img_szd = img_szd.permute(2, 0, 1).unsqueeze(0)

# Carga el modelo y muévelo a la GPU
model = YOLO("D:/edurb/Escritorio/Escuela/MODULAR/Program/STA/MODEL/weights/best.pt").to(device)

# Mueve la imagen a la GPU
img_szd = img_szd.to(device)

# Realiza la predicción
pred = model.predict(img_szd)[0]
pred = pred.plot()

# Guarda la imagen de salida
cv2.imwrite(f"frame_255.png", pred.cpu())

TypeError: torch.finfo() requires a floating point input type. Use torch.iinfo to handle 'torch.finfo'

In [8]:
img = cv2.imread("D:/edurb/Escritorio/Escuela/MODULAR/Program/STA/MODEL/frame_255.jpg")
img_szd = cv2.resize(img, (640, 480))

model = YOLO("D:/edurb/Escritorio/Escuela/MODULAR/Program/STA/MODEL/weights_6G_10B/best.pt")

pred = model.predict(img_szd)[0]
pred = pred.plot()

cv2.imwrite(f"frame_255.png", pred)


0: 480x640 6 vehicle5s, 1 vehicle8, 231.9ms
Speed: 3.0ms preprocess, 231.9ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)


True

**Inferencia sobre video**

In [6]:
model = YOLO("weights/best.pt")

video_path = "video1.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
device = None
device = "cuda:0"

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame_count += 1
        frame = cv2.resize(frame, (640, 480))

        # if frame_count % 1 == 0:
        results = model.predict(frame, device=device)
        # Procesa los resultados
        boxes = results[0].boxes  # Objeto Boxes para las salidas de bbox
        num_vehicles = len(boxes)  # Cantidad de vehículos detectados
        print(f"Se detectaron {num_vehicles} vehículos en el frame.")
        # Muestra la imagen con las bounding boxes
        img_with_boxes = results[0].plot()
        cv2.imshow('Frame', img_with_boxes)

        # Rompe el ciclo si se presiona 'q'
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 6 truck_ss, 39.0ms
Speed: 3.3ms preprocess, 39.0ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)
Se detectaron 6 vehículos en el frame.

0: 480x640 5 truck_ss, 16.7ms
Speed: 2.2ms preprocess, 16.7ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)
Se detectaron 5 vehículos en el frame.

0: 480x640 5 truck_ss, 19.2ms
Speed: 1.5ms preprocess, 19.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
Se detectaron 5 vehículos en el frame.

0: 480x640 5 truck_ss, 19.9ms
Speed: 1.0ms preprocess, 19.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Se detectaron 5 vehículos en el frame.

0: 480x640 5 truck_ss, 20.7ms
Speed: 1.2ms preprocess, 20.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Se detectaron 5 vehículos en el frame.

0: 480x640 3 truck_ss, 15.7ms
Speed: 1.0ms preprocess, 15.7ms inference, 5.4ms postprocess per image at shape (1, 3, 480, 640)
Se detectaron 3 vehículos en el frame

In [ ]:

# model = YOLO("weights/best.pt")
model = YOLO("YOLOV8/yolov8l.pt")
video_path = "video1.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
device = None
device = "cuda:0"

# Tamaño del POI deseado
poi_width, poi_height = 640, 640

# Calcula las coordenadas para recortar el POI centrado
ret, frame = cap.read()
h, w, _ = frame.shape
poi_start_x = max(0, (w - poi_width) // 2)
poi_start_y = max(0, (h - poi_height) // 2)
poi_end_x = poi_start_x + poi_width
poi_end_y = poi_start_y + poi_height

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame_count += 1
        frame = cv2.resize(frame, (640, 480))

        # Recorta el punto de interés
        # frame_poi = frame[poi_start_y:poi_end_y, poi_start_x:poi_end_x]
        
        if frame_count % 1 == 0:
            results = model.predict(frame, device=device, verbose=True, conf=0.7, vid_stride=1, stream_buffer=False, show=False, agnostic_nms=True)
            # Procesa los resultados
            boxes = results[0].boxes  # Objeto Boxes para las salidas de bbox
            num_vehicles = len(boxes)  # Cantidad de vehículos detectados
            # print(f"Se detectaron {num_vehicles} vehículos en el frame.")
            
            # Muestra la imagen con las bounding boxes
            img_with_boxes = results[0].plot()
            cv2.imshow('Frame', img_with_boxes)
            
        # Rompe el ciclo si se presiona 'q'
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        elif cv2.waitKey(25) & 0xFF == ord('p'): # Presiona 'p' para pausar el video
            cv2.waitKey(0)
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
# importar threading para correr el video en un hilo separado, luego cada cierto tiempo, extraer un frame y realizar la detección
import threading
import os
import time

import numpy as np
import cv2
from ultralytics import YOLO
import torch

In [2]:

stop_thread = False
# Crear el modelo y el hilo de inferencia
model = YOLO("YOLOV8/yolov8l.pt")
device = "cuda:0"

# Función para leer el video y realizar la detección
def is_img_ready():
    ruta_imagen = os.path.join("frame_out.jpg")
    return os.path.isfile(ruta_imagen) 

def detect_frame(model, device = "cuda:0"):
    global stop_thread
    while not stop_thread:
        if is_img_ready():
            img = cv2.imread("frame_out.jpg")
            results = model.predict(img, device=device, verbose=True, conf=0.7, vid_stride=1, stream_buffer=False, show=True, agnostic_nms=True)
            os.remove("frame_out.jpg")
        elif stop_thread:
            break
        else:
            time.sleep(0.1)

inference_thread = threading.Thread(target=detect_frame, args=(model, device))

def read_video(video_path, dims = (640, 480)):
    global stop_thread
    stop_thread = False
    inference_thread.start()
    
    cap = cv2.VideoCapture(video_path)
    
    frame_count = 0
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame_count += 1
            frame = cv2.resize(frame, dims)
            
            # Mostrar el frame
            cv2.imshow('Frame', frame)
            
            if frame_count % 30 == 0: 
                cv2.imwrite("frame_out.jpg", frame)
                
            # Rompe el ciclo si se presiona 'q'
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()
    stop_thread = True
    inference_thread.join()




In [3]:
read_video("video1.mp4")


0: 480x640 4 cars, 180.1ms
Speed: 3.0ms preprocess, 180.1ms inference, 78.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 17.3ms
Speed: 2.5ms preprocess, 17.3ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 cars, 14.2ms
Speed: 1.0ms preprocess, 14.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 14.6ms
Speed: 1.0ms preprocess, 14.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 cars, 16.6ms
Speed: 2.0ms preprocess, 16.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 cars, 1 truck, 14.1ms
Speed: 1.0ms preprocess, 14.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 cars, 36.8ms
Speed: 2.0ms preprocess, 36.8ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 44.1ms
Speed: 1.0ms preprocess, 44.1ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 6

: 

In [44]:
model = YOLO("YOLOV8/yolov8l.pt")
video_path = "video1.mp4"
results = model.predict(video_path, device=device, verbose=False, conf=0.7, vid_stride=4, stream_buffer=True, show=False, agnostic_nms=True)


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs



KeyboardInterrupt: 



# **Entrenamiento**

Verificar si hay GPU disponible

In [16]:
!nvidia-smi

Fri Mar  1 17:30:12 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.61                 Driver Version: 551.61         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:0A:00.0  On |                  N/A |
|  0%   47C    P5             25W /  285W |    3349MiB /  16376MiB |     18%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Instalar/Importar librerías

In [ ]:
!pip install ultralytics
# !pip install datasets

# from datasets import load_dataset
from ultralytics import YOLO
import numpy as np
import zipfile
import shutil
from google.colab import files

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.1/715.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


Incluír el set de datos de entrenamiento en formato YOLO

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/Colab_Notebooks/datasets.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/datasets')

Crear archivo de configuración del modelo en formato YAML

In [ ]:
# Frames de 640 x 480

yaml_content_vehicles = """
path: .
train: images/train
val: images/validation
test: images/test

# Classes
names:
  0: vehicle0
  1: vehicle1
  2: vehicle2
  3: vehicle3
  4: vehicle4
  5: vehicle5
  6: vehicle6
  7: vehicle7
  8: vehicle8
  9: vehicle9
  10: vehicle10
  11: vehicle11
  12: vehicle12

"""

with open('/content/train.yaml', 'w') as file:
    file.write(yaml_content_vehicles)

Configurar hiper parámetros para el entrenamiento

In [ ]:
_data="train.yaml"
_epochs=6
_imgsz=(640, 480)
_batch=20
_optimizer="Adam"

_model="yolov8m.pt"

Cargar el modelo a usar (n, s, m...)

In [ ]:
"""
Model    size       mAPval  Speed          params   FLOPs
        (pixels)    50-95   CPU ONNX        (M)     (B)

YOLOv8n	640	        37.3	 80.4	 	    3.2	    8.7
YOLOv8s	640	        44.9	 128.4	 	    11.2	28.6
YOLOv8m	640	        50.2	 234.7	 	    25.9	78.9
YOLOv8l	640	        52.9	 375.2	 	    43.7	165.2
YOLOv8x	640	        53.9	 479.1	 	    68.2	257.8
"""

model = YOLO(_model)

Entrenar modelo

In [ ]:
model.train(data=_data, epochs=_epochs, imgsz=_imgsz, batch=_batch, optimizer=_optimizer)

Ultralytics YOLOv8.1.15 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=train.yaml, epochs=6, time=None, patience=50, batch=20, imgsz=(640, 480), save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 17.06 MiB is free. Process 2355 has 14.72 GiB memory in use. Of the allocated memory 14.45 GiB is allocated by PyTorch, and 46.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Descargar modelo entrenado

In [ ]:
shutil.make_archive('/content/weights', 'zip', '/content/runs/detect/train/weights') # Comprimir archivo de pesos en drive
# files.download('/content/weights.zip')

'/content/weights.zip'

Copiar modelo entrenado a drive

In [ ]:
shutil.copy('/content/weights.zip', '/content/drive/MyDrive/Colab_Notebooks/weights2.zip')

'/content/drive/MyDrive/Colab_Notebooks/weights2.zip'

Pruebas de inferencia

In [ ]:
import cv2
img = cv2.imread("/content/frame_107.jpg")
img_szd = cv2.resize(img, (640, 480))

model = YOLO("/content/runs/detect/train/weightsbest.pt")

pred = model.predict(img_szd)[0]
pred = pred.plot()

cv2.imwrite(f"frame_107.png", pred)


0: 480x640 5 vehicle5s, 29.1ms
Speed: 1.6ms preprocess, 29.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)


True